# Using CODEX Multiplexed Imaging Data as Input (25% PD1+ T cell)

This notebook was used to import CODEX multiplexed imaging data for the 25% PD1+ T cell treated condition to initialize a tumor-tcell experiment

In [1]:
#Import Packages
#Needed for moving to output
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as pl
import itertools
from collections import Counter

#Import other needed packages
from vivarium.library.units import units, remove_units
from tumor_tcell.experiments.main import tumor_tcell_abm
from tumor_tcell.experiments.main import plots_suite
from tumor_tcell.experiments.main import get_tcells
from tumor_tcell.experiments.main import get_tumors
from tumor_tcell.library.phylogeny import get_phylogeny

Import the data needed from a CODEX multiplexed experiment (this experiment output only contains necessary input columns to the tumor-tcell)

In [3]:
#Read in the data from parent directory
analysis_dir = '/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/'
experiment_id = '2HC/'
experiment_dir = analysis_dir+experiment_id
os.chdir(experiment_dir)

#Read in the data from parent directory
df_2HC_exp = pd.read_csv('2HCTcell_exp_1of4.csv')
df_2HC_exp.rename(columns = {"Unnamed: 0": "cell_index"},  inplace = True) 
df_2HC_exp.set_index('cell_index', inplace = True)
df_2HC_exp

'/mnt/c/Users/akoya-stanford/Python_Code/tumor-tcell/out/analysis/2HC'

This block of code establishes intial parameters for the tumor_tcell_abm simulation:
* bounds - the size of the experimental space in microns for x,y
* Y pos - the column number for the y column in dataframe imported
* X pos - the column number for the x column in dataframe imported
* H2Kb pos - the column number for the H2Kb/MHCI column in dataframe imported (used for tumor cells)
* PD1 - the column number for the PD1 column in dataframe imported (used for T cells)
* PDL1 - the column number for the PD1 column in dataframe imported (used for tumor cells)
* Celltype - the column number for the cell type identified from the CODEX data
* IFNg_con - the column number for thedefault level of IFNg for all cells
* tumor_diameter - control tumor diameter in simulation
* tcell_diameter - control t cell diameter in simulation
It will loop through the imported dataframe and establish a structure of dictionaries that can be used as input to the tumor_tcell_abm simulation 

In [5]:
#Set bounds
bounds = [1800,1800]

#Create a copy
df_2HC_exp1 = df_2HC_exp.copy()

#Column Number for iloc iterating
Ypos = 0
Xpos = 1
H2Kb = 3
PD1 = 4
PDL1 = 5
Celltype = 7
IFNg_con = 8 

tumor_diameter = 15
tcell_diameter = 7.5

tumors = {}
IFNg = {}
tcells = {}

for i in range(len(df_2HC_exp1)):
  if df_2HC_exp1.iloc[i, Celltype] == 'PDL1p' or df_2HC_exp1.iloc[i, Celltype] == 'PDL1n':
    tumors.update({
        'tumor_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tumor',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PDL1': df_2HC_exp1.iloc[i, PDL1],
            'MHCI': df_2HC_exp1.iloc[i, H2Kb],
            'diameter': tumor_diameter*units.um,
        }
    })
    
    IFNg.update({
        df_2HC_exp1.index[i] : {
            'amount': df_2HC_exp1.iloc[i, IFNg_con],
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

  else: 
    tcells.update({
        'tcell_'+ str(df_2HC_exp1.index[i]) : {
            'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
            'type': 'tcell',
            'cell_state': df_2HC_exp1.iloc[i, Celltype],
            'PD1': df_2HC_exp1.iloc[i, PD1],
            'diameter': tcell_diameter*units.um,
        }
    })
    IFNg.update({
        df_2HC_exp1.index[i]: {
        'amount': df_2HC_exp1.iloc[i, IFNg_con],
        'location': [df_2HC_exp1.iloc[i, Xpos]*units.um,df_2HC_exp1.iloc[i, Ypos]*units.um],
        }
    })

# print the first tcell
tcell_ids = list(tcells.keys())
print(tcells[tcell_ids[0]])

{'tcell_17564': {'location': [60.497795898437516 <Unit('micrometer')>,
   413.5967653808593 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 27198.82947357501,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17565': {'location': [252.2080484619141 <Unit('micrometer')>,
   552.7608481445313 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 19922.6654816716,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17566': {'location': [274.46803002929687 <Unit('micrometer')>,
   603.99879296875 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 44280.58224949624,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17567': {'location': [266.0270865478516 <Unit('micrometer')>,
   585.8332099609373 <Unit('micrometer')>],
  'type': 'tcell',
  'cell_state': 'PD1p',
  'PD1': 39618.41980208061,
  'diameter': 7.5 <Unit('micrometer')>},
 'tcell_17570': {'location': [389.5870417480469 <Unit('micrometer')>,
   578.585440185547 <Unit('

Here tumor_tcell_abm can directly be called, initialized with the two dictionaries created (tumors, tcells), set for the time and bounds. 
Also, parameters like halt_threshold (number of cells to stop the simulation) and emit_step (how often to output data from the simulation in second). 
emitter=database enables storing the data in a database for future retrieval and usually is not changed. 

In [6]:
#using experimental CODEX data
data = tumor_tcell_abm(tumors=tumors, 
                       tcells=tcells, 
                       total_time=259200, 
                       bounds=[b*units.um for b in bounds], 
                       halt_threshold=10000,
                       emit_step=600,
                       emitter='database',)
data;

Initializing experiment tumor_tcell_20210716.142733


100%|██████████| 432/432 [42:31:28<00:00, 354.37s/it]   


Completed in 153088.51 seconds
